In [ ]:
#steps

# Import large Mt5 model
# bring in dataset
# Fine tune
# Evaluate

In [2]:
##Bring in dataset

from datasets import load_dataset

dataset = load_dataset("squad_es",'v1.1.0')


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87595 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad_es downloaded and prepared to /root/.cache/huggingface/datasets/squad_es/v1.1.0/1.1.0/bcada4f600192451443b95e24f609325705c5185b8aad97bffa8bc3784a867ad. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

NameError: name 'datset' is not defined

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87595
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [10]:
dataset['train']['answers']

[{'text': ['Santa Bernadette Soubirous'], 'answer_start': [572]},
 {'text': ['una estatua de cobre de Cristo'], 'answer_start': [218]},
 {'text': ['el edificio principal'], 'answer_start': [88]},
 {'text': ['un lugar mariano de oración y reflexión'], 'answer_start': [430]},
 {'text': ['una estatua dorada de la Virgen María'], 'answer_start': [114]},
 {'text': ['de una página en septiembre de 1876'], 'answer_start': [295]},
 {'text': ['dos veces'], 'answer_start': [504]},
 {'text': ['The Observer'], 'answer_start': [675]},
 {'text': ['tres periódicos'], 'answer_start': [182]},
 {'text': ['1987'], 'answer_start': [1015]},
 {'text': ['Roma'], 'answer_start': [117]},
 {'text': ['Seminario de Moreau'], 'answer_start': [151]},
 {'text': ['Old College'], 'answer_start': [254]},
 {'text': ['Sacerdotes y hermanos retirados'], 'answer_start': [374]},
 {'text': ['Premio Buechner para la'], 'answer_start': [723]},
 {'text': ['programas de doble titulación de'], 'answer_start': [482]},
 {'text': ['

In [12]:
import torch
from transformers import MT5Tokenizer

# Initialize the tokenizer
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-base")

# Define a function to create a batch of data
def create_data_batch(n, tokenizer):
    # Create dummy data
    titles = ["This is title " + str(i) for i in range(n)]
    contexts = ["This is context " + str(i) for i in range(n)]
    questions = ["This is question " + str(i) + "?" for i in range(n)]
    answers = ["This is answer " + str(i) + "." for i in range(n)]

    # Preprocess the data
    input_str = [title + " " + context + " " + question for title, context, question in zip(titles, contexts, questions)]
    inputs = tokenizer(input_str, truncation=True, padding='max_length', max_length=512, return_tensors="pt")
    
    answer_str = answers
    labels = tokenizer(answer_str, truncation=True, padding='max_length', max_length=128, return_tensors="pt").input_ids
    
    # Return a dictionary of tensors
    return {
        'input_ids': inputs.input_ids, 
        'attention_mask': inputs.attention_mask,
        'labels': labels
    }

# Create a batch of training data
train_data = create_data_batch(64, tokenizer)

# Create a batch of validation data
validation_data = create_data_batch(16, tokenizer)


In [24]:
# Extract one example from each tensor
sample = {
    'input_ids': train_data['input_ids'][0],
    'attention_mask': train_data['attention_mask'][0],
    'labels': train_data['labels'][0]
}

assert isinstance(sample, dict), "Each item in dataset must be a dictionary"
assert 'input_ids' in sample, "input_ids must be in the dataset"
assert 'attention_mask' in sample, "attention_mask must be in the dataset"
assert 'labels' in sample, "labels must be in the dataset"

print("An example in your dataset appears to be correctly structured.")


An example in your dataset appears to be correctly structured.


In [26]:
# Convert train_data from dict of lists to list of dicts
validation_data = [dict(zip(validation_data, t)) for t in zip(*validation_data.values())]

# Now train_data[0] should work, let's print it
print(validation_data[0])

{'input_ids': tensor([ 1494,   339, 10331,   497,  1494,   339, 19730,   497,  1494,   339,
         7680,   497,   291,     1,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

In [25]:
# Convert train_data from dict of lists to list of dicts
train_data = [dict(zip(train_data, t)) for t in zip(*train_data.values())]

# Now train_data[0] should work, let's print it
print(train_data[0])


{'input_ids': tensor([ 1494,   339, 10331,   497,  1494,   339, 19730,   497,  1494,   339,
         7680,   497,   291,     1,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

In [28]:
# replace 'train_dataset' with your dataset variable
sample = validation_data[0]

assert isinstance(sample, dict), "Each item in dataset must be a dictionary"
assert 'input_ids' in sample, "input_ids must be in the dataset"
assert 'attention_mask' in sample, "attention_mask must be in the dataset"
assert 'labels' in sample, "labels must be in the dataset"

print("Your dataset appears to be correctly structured for Trainer.")


Your dataset appears to be correctly structured for Trainer.


## Fine Tuning

In [29]:
from transformers import MT5ForConditionalGeneration, Trainer, TrainingArguments

# Initialize the model
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-base")

# Initialize the training arguments
training_args = TrainingArguments(
    output_dir="./results",          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

# Define the compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# Initialize the Trainer
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_data,            # training dataset
    eval_dataset=validation_data,         # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

# Train the model
trainer.train()


loading configuration file https://huggingface.co/google/mt5-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/5ebfd830555547194403d6803baa127970de59b443c04b7a1a60b16a97ed3958.b589da7dac64196f9764abaf2c4c7e507cec8b14b96da3ef270d924f155062de
Model config MT5Config {
  "_name_or_path": "/home/patrick/hugging_face/t5/mt5-base",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Token

Step,Training Loss
10,51.877200




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=12, training_loss=51.85478909810384, metrics={'train_runtime': 11.0849, 'train_samples_per_second': 17.321, 'train_steps_per_second': 1.083, 'total_flos': 230216857288704.0, 'train_loss': 51.85478909810384, 'epoch': 3.0})